In [11]:
import unittest
import warnings
import numpy as np
import logging as log
import subprocess as sp
import random as random
import parmed as pmd
import paprika
from paprika.openmm_simulate import *
from paprika.restraints import *
from paprika.utils import HAS_OPENMM, decompose_openmm_energy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
forces = []
for i in [0, 1]:

    my_simulation = OpenMM_GB_simulation()
    my_simulation.topology = '../test/cb6-but/vac.topo'
    my_simulation.md['platform'] = 'Reference'
    my_simulation.md['coordinates'] = '../test/cb6-but/vac.crds'
    my_simulation.md['steps'] = 10
    my_simulation.md['output'] = 'tmp.nc'
    simulation, system = my_simulation.setup_system(my_simulation.md, seed=42)
    
    if i:
        print('Adding restraints...')

        restraint = DAT_restraint()
        restraint.structure_file = my_simulation.topology
        restraint.mask1 = ':BUT@C1'
        restraint.mask2 = ':CB6@C1'
        restraint.attach['target'] = 15.0
        restraint.attach['num_windows'] = 4
        restraint.attach['fc_initial'] = 0.0
        restraint.attach['fc_final'] = 30.0
        restraint.auto_apr = False
        restraint.initialize()

        system = my_simulation.add_openmm_restraints(system, [restraint], phase='attach', window=3)
        # log.debug(system.getForces())

       
    asdf = my_simulation.run_md(simulation, seed=42, save=True)
    log.debug(system.getForces())
    omm_f = my_simulation.context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(u.kilocalories_per_mole/u.angstroms)
    forces.append(omm_f)

    state = my_simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    print(energy)
    # del my_simulation
print(forces[1] - forces[0])

2018-02-01 04:24:28 PM Integrator random number seed: 42
2018-02-01 04:24:28 PM Platform: Reference
2018-02-01 04:24:28 PM Velocity random number seed: 42
2018-02-01 04:24:28 PM Running OpenMM MD...
2018-02-01 04:24:28 PM MD completed.
2018-02-01 04:24:28 PM [<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fb86028a030> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fb86028a3c0> >, <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fb86028a6c0> >, <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fb86028a600> >, <simtk.openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7fb86028a510> >, <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fb86028a240> >]


AttributeError: 'OpenMM_GB_simulation' object has no attribute 'context'

In [89]:
forces = []
for i in [0, 1]:
    prmtop = app.AmberPrmtopFile('../test/cb6-but/vac.topo')
    inpcrd = app.AmberInpcrdFile('../test/cb6-but/vac.crds')
    system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
    integrator = mm.LangevinIntegrator(300, 1/u.picosecond,  0.0005*u.picoseconds)
    simulation = app.Simulation(prmtop.topology, system, integrator)
    simulation.context.setPositions(inpcrd.positions)
    
    if i:
        print('Adding restraints...')
        bond_restraint = mm.CustomBondForce('k * (r - r_0)**2')
        bond_restraint.addPerBondParameter('k')
        bond_restraint.addPerBondParameter('r_0')

        r_0 = 1.5 * unit.nanometers
        k = 100 * unit.kilojoules_per_mole / unit.nanometers ** 2
        bond_restraint.addBond(1, 100, [k, r_0])
        system.addForce(bond_restraint)

        log.debug(system.getForces())

    simulation.context.setVelocitiesToTemperature(300 * unit.kelvin, 42)
    simulation.step(10)

    omm_f = simulation.context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(u.kilocalories_per_mole/u.angstroms)
    forces.append(omm_f)

    state = simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    print(energy)
    # del my_simulation
print(forces[1] - forces[0])

2018-02-01 02:50:43 PM [<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fb861e94ab0> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fb861cc78a0> >, <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fb861cc7a50> >, <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fb861cc7c90> >, <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fb861cc7ed0> >, <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7fb861cc7cc0> >]


-640.8881309714086
Adding restraints...
-639.4468836853105
[[ -1.03875795e+00  -2.20976841e-01  -6.53496455e-01]
 [ -1.15839643e+00  -2.47640044e+00   5.57668278e+00]
 [ -4.74130096e-01   5.69399718e-01   2.33002875e+00]
 [  3.56357431e+00  -9.31076466e-01  -1.70566915e+00]
 [  1.05287012e+00   3.14255016e-01  -9.20928991e-01]
 [  9.91901784e-01   3.67884408e+00  -3.95002330e-01]
 [  4.33472429e+00  -2.00538933e+00  -2.71655475e+00]
 [ -5.28012731e+00   1.34073187e+00  -3.10420488e+00]
 [ -3.74116691e+00  -2.34616693e+00   3.99144669e-02]
 [  2.08008135e-01   5.45337483e-03   8.04639208e+00]
 [  2.83169147e+00  -5.32787597e-01   2.02554090e+00]
 [ -2.51123371e+00  -6.36686227e-01  -4.31209380e+00]
 [ -7.16820832e-01   7.02752985e-01  -2.07366948e+00]
 [  9.96931628e-01   6.80550665e-01   2.78764343e+00]
 [  2.68111710e-01  -8.44974735e-01   1.20101045e-01]
 [  1.59596646e+00   3.00220806e+00   1.28244577e+00]
 [ -2.48783280e-01   3.38665303e-01  -4.37854223e+00]
 [  4.68311504e-01  -7.

In [15]:
print(energy)

-770.6935717188372


In [24]:
state = my_simulation.context.getState(getEnergy=True, getForces=True)

In [30]:
energy_decomposition(parm, my_simulation.context)

defaultdict(float,
            {'angle': 139.9506130443077,
             'bond': 13.609467849964242,
             'dihedral': 59.56193375504047,
             'nonbonded': -983.8155863681499,
             'total': -770.6935717188376})

In [72]:
forces = my_simulation.context.getSystem().getForces()

In [73]:
forces

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fb8623bb2d0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fb8623bbf60> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fb8623bbae0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fb8623bbba0> >,
 <simtk.openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7fb8623bbed0> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fb8623bba50> >,
 <simtk.openmm.openmm.CustomCentroidBondForce; proxy of <Swig Object of type 'OpenMM::CustomCentroidBondForce *' at 0x7fb861e94db0> >]

In [42]:
dir(parm)

['ANGLE_FORCE_GROUP',
 'BOND_FORCE_GROUP',
 'CMAP_FORCE_GROUP',
 'DIHEDRAL_FORCE_GROUP',
 'IMPROPER_FORCE_GROUP',
 'LJ_depth',
 'LJ_radius',
 'LJ_types',
 'NONBONDED_FORCE_GROUP',
 'OUT_OF_PLANE_BEND_FORCE_GROUP',
 'PI_TORSION_FORCE_GROUP',
 'RB_TORSION_FORCE_GROUP',
 'STRETCH_BEND_FORCE_GROUP',
 'TORSION_TORSION_FORCE_GROUP',
 'TRIGONAL_ANGLE_FORCE_GROUP',
 'UREY_BRADLEY_FORCE_GROUP',
 '__add__',
 '__bool__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_force_to_system',
 '_add_missing_13_14',
 '_add_standard_flags',
 '_box',
 '_ch

In [44]:
state = my_simulation.context.getState(getEnergy=True)

In [60]:
print('{0:5.5f}'.format(state.getPotentialEnergy()._value))

-3224.58190


# A Swails test

In [28]:
from parmed import Structure, topologyobjects, load_file
from parmed.exceptions import ParameterError
from parmed.amber import AmberParm, ChamberParm, Rst7
from parmed.openmm import load_topology, energy_decomposition_system
import parmed.unit as u

def energy_decomposition(parm, context):
    from parmed.openmm.utils import energy_decomposition
    ret = defaultdict(float)
    for key, val in energy_decomposition(parm, context).items():
        ret[key] = val
    return ret

parm = AmberParm('../test/cb6-but/vac.topo', '../test/cb6-but/vac.crds')
parm.combining_rule
system = parm.createSystem() # Default, no cutoff
integrator = mm.VerletIntegrator(1.0*u.femtoseconds)
CPU = mm.Platform.getPlatformByName('CPU')
sim = app.Simulation(parm.topology, system, integrator, platform=CPU)
sim.context.setPositions(parm.positions)
energies = energy_decomposition(parm, sim.context)

In [18]:
energies

defaultdict(float,
            {'angle': 109.44915017870449,
             'bond': 11.411584696403484,
             'dihedral': 52.95780165980911,
             'nonbonded': -863.3976443701897,
             'total': -689.5791078352727})

In [21]:
decompose_openmm_energy(parm, my_simulation.context)

{'angle': 160.53479464512478,
 'bond': 28.537264874081288,
 'dihedral': 66.19491634127635,
 'nonbonded': -978.8701450751796,
 'total': -723.6031692146971}

In [74]:
omm_f = my_simulation.context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(u.kilocalories_per_mole/u.angstroms)

In [27]:
omm_e = pmd.openmm.energy_decomposition_system(parm, system)

In [29]:
for name, oe in omm_e:
    if name == 'HarmonicBondForce':
        term, ae = 'Bond', e.bond
    elif name == 'HarmonicAngleForce':
        term, ae = 'Angle', e.angle
    elif name == 'PeriodicTorsionForce':
        term, ae = 'Dihedral', e.dihedral
    elif name == 'NonbondedForce':
        term, ae = 'Nonbonded', e.elec+e.vdw+e.elec_14+e.vdw_14
    elif name in ('CustomGBForce', 'GBSAOBCForce'):
        term, ae = 'GB', e.gb
    else:
        continue
    total += oe
    print('%-20s | %15.6f | %15.6f' % (term, ae, oe))
print('%-20s | %15.6f | %15.6f' % ('Total', e.tot, total))
print('-'*56)
print('-'*56)

NameError: name 'e' is not defined

In [30]:
omm_e

[('HarmonicBondForce', 11.411568916467006),
 ('HarmonicAngleForce', 109.4491824325856),
 ('PeriodicTorsionForce', 52.95780213733279),
 ('NonbondedForce', -863.3975139240347),
 ('CMMotionRemover', 0.0)]

In [32]:
system.

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f0cb5f297b0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f0cb5f29b40> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f0cb5f29de0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f0cb5f294b0> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7f0cb5f299f0> >]

In [75]:
omm_f

array([[  5.14730713e+00,  -4.49318019e+01,  -9.02148899e+00],
       [ -3.48326901e+00,   5.65181703e+00,  -3.83288455e+00],
       [ -3.44723691e+00,   3.52626736e+01,   1.92801105e+01],
       [ -1.85871839e+01,   1.27614088e+01,  -6.10363103e+00],
       [  6.86853841e-01,   1.89601375e+00,  -7.01208848e+00],
       [  1.10181206e+01,  -2.74130421e+00,  -1.17633966e+00],
       [ -1.38680337e+01,   2.19669273e+01,   2.51785660e+01],
       [  1.63927554e+01,   1.05661323e+01,   1.04091641e+01],
       [ -2.86474993e+01,  -8.81544153e+00,   9.86825291e+00],
       [ -5.47027596e+00,  -4.63402734e+01,  -7.76507851e+00],
       [ -8.35479128e+00,  -1.09225328e+00,  -1.83735993e+00],
       [  1.94824438e+01,  -2.89103968e+01,  -2.90174279e+01],
       [  3.41656494e+00,   1.06471033e+01,  -2.35079372e+01],
       [ -7.53008539e+00,   1.33530803e+01,   2.33926083e+01],
       [  8.15380750e+00,  -2.22672435e+00,  -1.99862255e+00],
       [  3.79333614e+00,  -2.98012121e-01,  -1.5275748

In [82]:
system

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7fb861cc7a20> >